<a href="https://colab.research.google.com/github/Dkepffl/Dacon/blob/main/HD_AI_Challenge/HD_FE_1023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **HD 현대 AI Chalenge**
- 구글 드라이브 어마운트를 통해 데이터 로드
- 10.22 기준 리더보드 최저 점수(45.6250669998) 코드


## **| 구글 드라이브 어마운트**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **| 필요한 라이브러리 로드**

In [ ]:
# import libraries
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import os

## **| 작업 폴더 변경 및 데이터 로드**

In [ ]:
os.chdir("/content/drive/MyDrive/HD현대 AI Challenge") # 각자 드라이브에 맞춰 변경
os.getcwd()

'/content/drive/MyDrive/HD현대 AI Challenge'

In [ ]:
train = pd.read_csv('Data/train.csv').drop('SAMPLE_ID', axis=1)
test = pd.read_csv('Data/test.csv').drop('SAMPLE_ID', axis=1)

In [ ]:
# 원본 데이터셋 별도 저장
train_proto = train.copy()
test_proto = test.copy()

## **| 데이터 전처리**

### **1. ATA 변수 처리**

In [ ]:
from datetime import datetime

# datetime 컬럼 처리
train['ATA'] = pd.to_datetime(train['ATA'])
test['ATA'] = pd.to_datetime(test['ATA'])

# datetime을 여러 파생 변수로 변환
for df in [train, test]:
  df['YEAR'] = df['ATA'].dt.year
  df['MONTH'] = df['ATA'].dt.month
  df['DAY'] = df['ATA'].dt.day
  df['HOUR'] = df['ATA'].dt.hour
  df['MINUTE'] = df['ATA'].dt.minute
  df['WEEKDAY'] = df['ATA'].dt.weekday

In [ ]:
train.drop('ATA', axis=1, inplace=True)
test.drop('ATA', axis=1, inplace=True)

### **2. 파생 변수 추가**

#### **선박 유형 별 선박 크기 관련 컬럼 추가**

In [ ]:
temp_train=train.copy()
temp_test=test.copy()

temp_train['SIZE']= temp_train['GT']+temp_train['BREADTH']+temp_train['DEADWEIGHT']+temp_train['DEPTH']+temp_train['DRAUGHT']+temp_train['LENGTH']
temp_test['SIZE']= temp_test['GT']+temp_test['BREADTH']+temp_test['DEADWEIGHT']+temp_test['DEPTH']+temp_test['DRAUGHT']+temp_test['LENGTH']

In [ ]:
ship_type1=train['SHIP_TYPE_CATEGORY'].unique().tolist()
ship_type2=test['SHIP_TYPE_CATEGORY'].unique().tolist()

In [ ]:
SHIP_dist_train=temp_train.groupby('SHIP_TYPE_CATEGORY')['SIZE'].agg(['mean','std']).reset_index()
SHIP_dist_test=temp_test.groupby('SHIP_TYPE_CATEGORY')['SIZE'].agg(['mean','std']).reset_index()

In [ ]:
for ship in ship_type1:
  train.loc[train['SHIP_TYPE_CATEGORY']==ship,'SIZE_MEAN'] = SHIP_dist_train.loc[SHIP_dist_train['SHIP_TYPE_CATEGORY']==ship,'mean'].values[0]
  train.loc[train['SHIP_TYPE_CATEGORY']==ship,'SIZE_STD'] = SHIP_dist_train.loc[SHIP_dist_train['SHIP_TYPE_CATEGORY']==ship,'std'].values[0]

for ship in ship_type2:
  test.loc[test['SHIP_TYPE_CATEGORY']==ship,'SIZE_MEAN'] = SHIP_dist_test.loc[SHIP_dist_test['SHIP_TYPE_CATEGORY']==ship,'mean'].values[0]
  test.loc[test['SHIP_TYPE_CATEGORY']==ship,'SIZE_STD'] = SHIP_dist_test.loc[SHIP_dist_test['SHIP_TYPE_CATEGORY']==ship,'std'].values[0]

#### **ATA_PO 별 DIST 통계량**

In [ ]:
port_list1=train['ARI_PO'].unique().tolist()
port_list2=test['ARI_PO'].unique().tolist()

In [ ]:
DIST_dist_train=train.groupby('ARI_PO')['DIST'].agg(['mean','std']).reset_index()
DIST_dist_test=train.groupby('ARI_PO')['DIST'].agg(['mean','std']).reset_index()

In [ ]:
for port in port_list1:
  train.loc[train['ARI_PO']==port,'DIST_MEAN'] = DIST_dist_train.loc[DIST_dist_train['ARI_PO']==port,'mean'].values[0]
  train.loc[train['ARI_PO']==port,'DIST_STD'] = DIST_dist_train.loc[DIST_dist_train['ARI_PO']==port,'std'].values[0]

for port in port_list2:
  test.loc[test['ARI_PO']==port,'DIST_MEAN'] = DIST_dist_test.loc[DIST_dist_test['ARI_PO']==port,'mean'].values[0]
  test.loc[test['ARI_PO']==port,'DIST_STD'] = DIST_dist_test.loc[DIST_dist_test['ARI_PO']==port,'std'].values[0]

#### **ATA_CO 별 LENGTH 통계량**

In [ ]:
co_list1=train['ARI_CO'].unique().tolist()
co_list2=test['ARI_CO'].unique().tolist()

In [ ]:
LENGTH_length_train=train.groupby('ARI_CO')['LENGTH'].agg(['mean','std']).reset_index()
LENGTH_length_test=train.groupby('ARI_CO')['LENGTH'].agg(['mean','std']).reset_index()

In [ ]:
for co in co_list1:
  train.loc[train['ARI_CO']==co,'LENGTH_MEAN'] = LENGTH_length_train.loc[LENGTH_length_train['ARI_CO']==co,'mean'].values[0]
  train.loc[train['ARI_CO']==co,'LENGTH_STD'] = LENGTH_length_train.loc[LENGTH_length_train['ARI_CO']==co,'std'].values[0]

for co in co_list2:
  test.loc[test['ARI_CO']==co,'LENGTH_MEAN'] = LENGTH_length_test.loc[LENGTH_length_test['ARI_CO']==co,'mean'].values[0]
  test.loc[test['ARI_CO']==co,'LENGTH_STD'] = LENGTH_length_test.loc[LENGTH_length_test['ARI_CO']==co,'std'].values[0]

#### **연도 별 DIST 통계량**

In [ ]:
YEAR_train=train.groupby('YEAR')['DIST'].agg(['mean','std']).reset_index()
YEAR_test=train.groupby('YEAR')['DIST'].agg(['mean','std']).reset_index()

In [ ]:
for year in range(2014, 2024, 1):
  train.loc[train['YEAR']==year,'YEAR_MEAN'] = YEAR_train.loc[YEAR_train['YEAR']==year,'mean'].values[0]
  train.loc[train['YEAR']==year,'YEAR_STD'] = YEAR_train.loc[YEAR_train['YEAR']==year,'std'].values[0]

for year in range(2014, 2024, 1):
  test.loc[test['YEAR']==year,'YEAR_MEAN'] = YEAR_test.loc[YEAR_test['YEAR']==year,'mean'].values[0]
  test.loc[test['YEAR']==year,'YEAR_STD'] = YEAR_test.loc[YEAR_test['YEAR']==year,'std'].values[0]

#### **ARI_CO별 Container 비율**

In [ ]:
temp1 = (train.groupby(['ARI_CO','SHIP_TYPE_CATEGORY'])['ARI_PO'].count() / train.groupby(['ARI_CO'])['ARI_PO'].count()).reset_index()
temp2 = (test.groupby(['ARI_CO','SHIP_TYPE_CATEGORY'])['ARI_PO'].count() / test.groupby(['ARI_CO'])['ARI_PO'].count()).reset_index()

In [ ]:
country1=train['ARI_CO'].unique().tolist()
country2=test['ARI_CO'].unique().tolist()

In [ ]:
for col in country1:
  if temp1.loc[(temp1['ARI_CO']==col)&(temp1['SHIP_TYPE_CATEGORY']=='Container')].shape == (0,3):
    train.loc[train['ARI_CO']==col,'PORTION_CO']=0
  else:
    train.loc[train['ARI_CO']==col,'PORTION_CO']=1-temp1.loc[(temp1['ARI_CO']==col)&(temp1['SHIP_TYPE_CATEGORY']=='Container')].values[0,2]

for col in country2:
  if temp2.loc[(temp2['ARI_CO']==col)&(temp2['SHIP_TYPE_CATEGORY']=='Container')].shape == (0,3):
    test.loc[test['ARI_CO']==col,'PORTION_CO']=0
  else:
    test.loc[test['ARI_CO']==col,'PORTION_CO']=1-temp2.loc[(temp2['ARI_CO']==col)&(temp2['SHIP_TYPE_CATEGORY']=='Container')].values[0,2]

#### **평일/주말 컬럼**

In [ ]:
train['HOLIDAY']=train['WEEKDAY'].apply(lambda x: 1 if x in [5,6] else 0)
test['HOLIDAY']=test['WEEKDAY'].apply(lambda x: 1 if x in [5,6] else 0)

#### **피크타임 컬럼**

In [ ]:
train['PEAK_TIME'] = train['HOUR'].apply(lambda x: 1 if x in range(14,21) else 0)
test['PEAK_TIME'] = test['HOUR'].apply(lambda x: 1 if x in range(14,21) else 0)

### **3. 카테고리 변수 라벨 인코딩**

In [ ]:
from sklearn.preprocessing import LabelEncoder
import bisect
from tqdm import tqdm

categorical_features = ['ARI_CO','ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER','FLAG']
encoders = {}

for feature in tqdm(categorical_features, desc="Encoding features"):
  le = LabelEncoder()
  train[feature] = le.fit_transform(train[feature].astype(str))
  le_classes_set = set(le.classes_)
  test[feature] = test[feature].map(lambda s: '-1' if s not in le_classes_set else s)
  le_classes = le.classes_.tolist()
  bisect.insort_left(le_classes, '-1')
  le.classes_ = np.array(le_classes)
  test[feature] = le.transform(test[feature].astype(str))
  encoders[feature] = le

Encoding features: 100%|██████████| 6/6 [00:01<00:00,  3.23it/s]


### **4. 미사용 컬럼 제거**

In [ ]:
co_matrix=train.corr()
correlations_with_x = co_matrix['CI_HOUR']
correlations_with_x.abs().sort_values(ascending=False)

CI_HOUR               1.000000
DIST                  0.189145
WEEKDAY               0.111632
SHIP_TYPE_CATEGORY    0.087101
LENGTH_MEAN           0.084063
DEADWEIGHT            0.077954
GT                    0.059661
DEPTH                 0.051760
BREADTH               0.043711
DRAUGHT               0.040309
SIZE_MEAN             0.040032
AIR_TEMPERATURE       0.035827
LENGTH                0.035334
DIST_STD              0.028731
YEAR                  0.021253
FLAG                  0.021199
PORT_SIZE             0.020635
PEAK_TIME             0.018626
V_WIND                0.017415
BN                    0.016283
SIZE_STD              0.013462
U_WIND                0.012193
LENGTH_STD            0.008949
MINUTE                0.008505
ID                    0.008185
DIST_MEAN             0.007737
HOUR                  0.007474
ARI_CO                0.005224
MONTH                 0.004286
SHIPMANAGER           0.003544
BUILT                 0.001756
ARI_PO                0.000459
DAY     

In [ ]:
train.drop(columns=['ID', 'SHIPMANAGER', 'FLAG', 'U_WIND', 'V_WIND', 'AIR_TEMPERATURE', 'BUILT','BN','DAY'], inplace=True)
test.drop(columns=['ID', 'SHIPMANAGER', 'FLAG', 'U_WIND', 'V_WIND', 'AIR_TEMPERATURE', 'BUILT','BN','DAY'], inplace=True)

### **5. 결측치 처리**

In [ ]:
train = train.fillna(-1)
test = test.fillna(-1)

### **6. 스케일링**

In [ ]:
from sklearn.preprocessing import StandardScaler

scl = StandardScaler()

train_scld = scl.fit_transform(train.drop('CI_HOUR', axis=1))
train_scld = pd.DataFrame(train_scld, columns=train.drop('CI_HOUR', axis=1).columns)

test_scld = scl.transform(test)
test_scld = pd.DataFrame(test_scld, columns=test.columns)

### **학습/검증 데이터셋 생성**

In [ ]:
from sklearn.model_selection import train_test_split
X = train_scld
Y = train["CI_HOUR"]
x_train, x_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2, random_state=42)

### **Catboost 모델링**

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.5 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostRegressor

cbc = CatBoostRegressor(verbose=1000, iterations=20000, objective='MAE', use_best_model=True)
cbc.fit(x_train, y_train, eval_set=(x_valid, y_valid))

0:	learn: 60.7819000	test: 59.7183715	best: 59.7183715 (0)	total: 158ms	remaining: 52m 49s
1000:	learn: 46.4697375	test: 45.6538143	best: 45.6538143 (1000)	total: 1m 41s	remaining: 32m 12s
2000:	learn: 45.6074614	test: 44.8824704	best: 44.8824704 (2000)	total: 3m 12s	remaining: 28m 54s
3000:	learn: 45.4264368	test: 44.7394869	best: 44.7382109 (2240)	total: 4m 42s	remaining: 26m 41s
4000:	learn: 45.4263872	test: 44.7394657	best: 44.7382109 (2240)	total: 6m 10s	remaining: 24m 40s
5000:	learn: 45.4263685	test: 44.7394467	best: 44.7382109 (2240)	total: 7m 38s	remaining: 22m 55s
6000:	learn: 45.4263515	test: 44.7394383	best: 44.7382109 (2240)	total: 9m 6s	remaining: 21m 15s
7000:	learn: 45.4263394	test: 44.7394238	best: 44.7382109 (2240)	total: 10m 38s	remaining: 19m 45s
8000:	learn: 45.4263260	test: 44.7394090	best: 44.7382109 (2240)	total: 12m 5s	remaining: 18m 8s
9000:	learn: 45.4263110	test: 44.7393941	best: 44.7382109 (2240)	total: 13m 33s	remaining: 16m 34s
10000:	learn: 45.4261165	te

In [ ]:
importances = cbc.get_feature_importance(type='PredictionValuesChange')
feature_importances = pd.Series(importances, index=X.columns).sort_values()

In [ ]:
feature_importances

MINUTE                 0.031211
ATA_LT                 0.076477
DRAUGHT                0.147854
MONTH                  0.186279
HOUR                   0.356083
DEPTH                  0.384143
YEAR_STD               0.601943
SIZE_MEAN              0.969893
YEAR_MEAN              1.115096
SIZE_STD               1.692967
YEAR                   1.724651
ARI_PO                 2.119798
SHIP_TYPE_CATEGORY     2.324926
LENGTH_MEAN            2.815708
DIST_MEAN              3.068466
LENGTH_STD             3.371962
DIST_STD               3.576187
DEADWEIGHT             3.667050
PORT_SIZE              4.285270
BREADTH                4.660115
GT                     4.934791
WEEKDAY                5.284405
ARI_CO                 5.680265
LENGTH                13.002847
DIST                  33.921614
dtype: float64

In [ ]:
from sklearn.metrics import mean_absolute_error

pred = cbc.predict(x_valid)
mean_absolute_error(pred, y_valid)

44.73821170001844

In [ ]:
# test 데이터셋에 대한 예측
final = cbc.predict(test_scld)

## **| 제출 파일 생성 및 저장**

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/HD현대 AI Challenge/Data/sample_submission.csv')
submit['CI_HOUR'] = final

In [ ]:
submit.to_csv('YOUN/sub_FE1023(6).csv', index=False) # 파일 이름 수정